<a href="https://colab.research.google.com/github/catherine2610/Prueba/blob/main/Prueba_Source_Meridian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### PYTHON/API TECHNICAL TEST 

Las siguientes API, contienen ingredientes de los cuales se debe consumir los sinónimos y estos deben ser clasificados, limpiados y cargados a un modelo SQL. 

1. Consuma la siguiente API:

```
https://rxnav.nlm.nih.gov/REST/allconcepts.json?tty=IN
```
Esta le permite ver una lista de ingredientes médicos 


In [5]:
import requests
import json
import re
import pandas as pd

url_ingredientes = 'https://rxnav.nlm.nih.gov/REST/allconcepts.json?tty=IN'  #configuramos la url

#solicitamos la información y guardamos la respuesta en response_1.
response_ingredientes = requests.get(url_ingredientes)  
json_ingredientes = response_ingredientes.json()  #convertimos la respuesta en dict

print(response_ingredientes)


<Response [200]>


In [7]:
lista_ingredientes = []

for nombre_ingredientes in json_ingredientes["minConceptGroup"]['minConcept']:

    if re.match(r'[a-z][\D]', nombre_ingredientes['name']):
        reemplazo = re.sub(r'[\(\)\-\[\]\,\.]',' ',nombre_ingredientes['name'])
        lista_ingredientes.append(reemplazo)
        
print(lista_ingredientes[:100])

['abacavir', 'abaloparatide', 'abarelix', 'abatacept', 'abciximab', 'abemaciclib', 'abetimus', 'abiraterone', 'acacia decurrens flower extract', 'acacia decurrens whole extract', 'acai berry extract', 'acai extract', 'acai fruit pulp', 'acai oil', 'acalabrutinib', 'acamprosate', 'acarbose', 'acebutolol', 'acecarbromal', 'aceclofenac', 'acefylline methylsilanol mannuronate', 'aceglutamide', 'acellular pertussis vaccine  inactivated', 'acemannan', 'acemetacin', 'acenocoumarol', 'acepromazine', 'aceprometazine', 'acesulfame', 'acetaldehyde', 'acetamide', 'acetamidoethoxyethanol', 'acetamidopropyl trimonium', 'acetaminophen', 'acetanilide', 'acetate', 'acetazolamide', 'acetiamine', 'acetic acid', 'acetic acid C 11', 'acetic anhydride', 'acetic monoethanolamide', 'acetohexamide', 'acetohydroxamic acid', 'acetone', 'acetophenazine', 'acetophenone', 'acetoxolone', 'acetyl arginine', 'acetyl carboxymethyl cocoyl glycine', 'acetyl cedrene', 'acetyl coenzyme A', 'acetyl decapeptide 3', 'acetyl d

2. Consuma la siguiente API: 

```
https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{ingrediente}/synonyms/json
```
Esta le permite ver la lista de los sinónimos de los ingredientes de la lista anterior, debe reemplazar {ingrediente}, con el nombre de la lista del paso 1.


In [ ]:
url_sinonimos = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{}/synonyms/json'
lista_sinonimos = []

for dct in json_ingredientes["minConceptGroup"]['minConcept']:
    try:
      
        response_sinonimos = requests.get(url_sinonimos.format(dct['name']))
        json_sinonimos = response_sinonimos.json() 
        lista_sinonimos.append(json_sinonimos)

    except:
        continue
    

Se guarda la lista en un archivo json:

In [ ]:
with open('data.json','w') as f:
  json.dump(lista_sinonimos,f,indent=4)

In [302]:
with open('data.json') as file:
    datos_sinonimos = json.load(file)

In [303]:
print(type(data))

<class 'list'>


In [304]:
print(datos_sinonimos[:30])

[{'InformationList': {'Information': [{'CID': 14650275, 'Synonym': ['97553-90-7', 'UNII-S78T1JQV6B', 'S78T1JQV6B', 'Methanol, (((1-methyl-2-(5-methyl-3-oxazolidinyl)ethoxy)methoxy)methoxy)-', '(((1-methyl-2-(5-methyl-3-oxazolidinyl)ethoxy)methoxy)methoxy)-methanol', '(((1-Methyl-2-(5-Methyl-3-Oxazolidinyl)Ethoxy)Methoxy)Methoxy)Methanol', '[[[1-Methyl-2-(5-methyl-3-oxazolidinyl)ethoxy]methoxy]methoxy]methanol', 'SCHEMBL1893826', 'DTXSID5034721', '(((1-Methyl-2-(5-methyl-3-oxazolidinyl)ethyoxy)methoxy)methoxy)methanol', 'Q27288768']}]}}, {'Fault': {'Code': 'PUGREST.NotFound', 'Message': 'No CID found', 'Details': ['No CID found that matches the given name']}}, {'InformationList': {'Information': [{'CID': 10857465, 'Synonym': ['(-)-Ambroxide', '6790-58-5', 'Ambroxan', 'Ambroxide', 'Ambrox', 'UNII-TD34B3O8M9', 'TD34B3O8M9', 'CHEBI:78307', '8alpha,12-Oxido-13,14,15,16-tetranorlabdane', 'FEMA No. 3471', '1,5,5,9-Tetramethyl-13-oxatricyclo[8.3.0.0(4,9)]tridecane', '(3aR,5aS,9aS,9bR)-3a,6,6,9

3. Estos sinónimos deben ser clasificados de la siguiente manera: 

●  Identificadores -> puede comenzar con algunas letras, pero contiene en su mayoría números, las letras no son relativas al ingredientes, Ex: MLS000069733 

●  Compuestos -> puede contener paréntesis, corchetes, números, que indican los compuestos del ingrediente, Ex: 4-Isobutyl-alpha-methylphenylacetic acid 

●  Traducciones -> contiene corchetes en los que indica el nombre del ingrediente en otro idioma, Ex: Ibuprofene [INN-French] 

● Ingrediente -> son aquellos que no están en las anteriores clasificaciones, pero tiene una relación con el ingrediente, Ex: Advil Liqui-Gels 

4. Su limpieza, depende de su clasificación: 

● Identificadores -> debe quedar solo el número de su identificación, Ex: MLS000069733 = 000069733 

● Compuestos -> debe quedar sin caracteres especiales, Ex: 
4-Isobutyl-alpha-methylphenylacetic acid = 4 Isobutyl alpha methylphenylacetic acid

● Traducciones -> se debe quitar el contenido de los corchetes, Ex: Ibuprofene [INN-French] = Ibuprofene 

Se define una función para clasificar y realizar la limpieza de los datos con expresiones regulares.

In [322]:
def clasificar(s):
  
  if re.match(r'^[a-zA-Z]+\d+$', s):
      clase='indicador'
      valor=re.findall(r'\d+', s)[0]

  elif re.match(r'^[a-zA-Z]+ \[.*\]$', s):
      clase='traducción'
      valor= s.split(' [')[0]


  elif re.match(r'^[\(|\[|\d]+\D+$', s):
      clase='compuesto' 
      valor=re.sub(r'[\(\)\-\[\]\{\}\.\,]',' ',s)

  elif re.match(r'^[\w]+\D+$', s):
      clase='ingrediente'
      valor = re.sub(r'[\(\)\-\[\]\,\.]',' ',s)


  return [clase,valor]


A continuación probamos la función de clasificación con los ejemplos:

In [323]:
clasificar ('MLS000069733')

['indicador', '000069733']

In [324]:
clasificar('Ibuprofene [INN-French]')

['traducción', 'Ibuprofene']

In [325]:
clasificar('(1-Cyanocyclohexyl)-acetic acid')

['compuesto', ' 1 Cyanocyclohexyl  acetic acid']

In [326]:
clasificar('Advil Liqui-Gels')

['ingrediente', 'Advil Liqui Gels']

5. Debe construir un modelo SQL que nos ayude a tener un mejor desempeño. Por favor adjunte el script SQL (Se adjunta en el Git)

6. Los datos de los ingredientes, sus sinónimos, clasificaciones y limpiezas deben ser cargados en este modelo, tenga en cuenta que será una tarea recurrente, escoja la mejor solución.

####Clasifiación

In [327]:
lista_indicador =[]
lista_traduccion =[]
lista_compuesto =[]
lista_ingrediente_sinonimo =[]

for lst in datos_sinonimos:
    try:
        for sinonimo in lst['InformationList']['Information'][0]['Synonym']:

            if (clasificar(sinonimo))[0]=='indicador':
                lista_indicador.append(clasificar(sinonimo)[1])

            elif (clasificar(sinonimo))[0]=='traducción':
                lista_traduccion.append(clasificar(sinonimo)[1])

            elif (clasificar(sinonimo))[0]=='compuesto':
                lista_compuesto.append(clasificar(sinonimo)[1])

            elif (clasificar(sinonimo))[0]=='ingrediente':
                lista_ingrediente_sinonimo.append(clasificar(sinonimo)[1])
            
        
    except:
        continue


In [328]:
print(lista_indicador)

['942690', '144673', '400', '497001', '27281064', '3', '1200706', '2036405', '27275662', '9717201', '401014240', '104', '207', '20556367', '27293196', '00149613', '3666', '101', '14663410', '10788', '378', '015899605', '30988741', '024350585', '85562417', '27293594', '07549', '818', '49244', '126518', '3299031', '3058321', '27254647', '27271394', '2', '11197', '1191', '5572303', '905673', '14035818', '3899231', '4118441', '423846', '27279034', '81047', '3291398', '27266252', '2211', '104', '161239', '17096041', '0010', '27280732', '00011109', '50973073', '70894993', '754952', '2', '00003463', '461', '1201235', '412', '7364', '10142864', '121175', '2912822', '12518166', '27283332', '2', '10', '130', '333328', '699', '60444', '054828', '1091256', '50410807', '00781', '807660', '00156359', '152846', '2104433', '22389514', '228', '9535', '2146115', '001333570', '000875281', '6218', '1230373']


In [330]:
df_indicador = pd.DataFrame(lista_indicador, columns=['Indicador'])
df_indicador

,Indicador
0,942690
1,144673
2,400
3,497001
4,27281064
...,...
85,2146115
86,001333570
87,000875281
88,6218


In [331]:
print(lista_compuesto[:50])

['    Ambroxide', ' 2 Carbethoxyethyl diethoxy methyl silane', ' 2 Carbethoxyethyl triethoxysilane', '2   hydroxymethyl amino ethanol', ' 3 Bromopropyl benzene', '3 Chloropropyltrimethoxysilane', '4  Trifluoromethoxy phenylurea', ' Dibutylamino methanol', ' Ethylenedinitrilo tetraacetonitrile', ' p Hydroxyphenyl dimethylsulfonium chloride', '8 Damascone', ' +  delta Tocopherol', '8 Methyltocol', '2 Hydroxypropyl benzoate', '4 Hydroxybutyl stearate', '6 Paradol', '1 BUTENE', '1 DECENE', '1 EICOSENE', '1 Hexadecene', '1 Methylhydantoin', '1 NAPHTHALENESULFONIC ACID', '1 NAPHTHOL', '1 o tetradecylglycerol', '1 Octacosanol', '1 octanol', '1 Phenylethanol', '1 propanol', '1 Tetracosanol', '10 Hydroxydecanoic acid', '10 Hydroxystearic acid', '10 Undecenal', '12R HEPE', '12 hydroxyoctadecanoate', '12 hydroxystearate', '12 hydroxystearic acid anion', '12 Oxooctadecanoic acid', '16 METHYLHEPTADECANOIC ACID', '17ALPHA HYDROXYPROGESTERONE', '17 Hydroxyprogesterone', '17 Hydroxystearic acid', '17 

In [344]:
df_compuesto = pd.DataFrame(lista_compuesto, columns=['Compuesto'])
df_compuesto = df_compuesto.rename_axis('Compuesto_Id').reset_index()
df_compuesto

,Compuesto_Id,Compuesto
0,0,Ambroxide
1,1,2 Carbethoxyethyl diethoxy methyl silane
2,2,2 Carbethoxyethyl triethoxysilane
3,3,2 hydroxymethyl amino ethanol
4,4,3 Bromopropyl benzene
...,...,...
595,595,2 Propylpentanoic acid
596,596,Eburnamonine
597,597,22 Oxovincaleukoblastine
598,598,+ Viridiflorol


In [345]:
print(lista_traduccion)

['Alatrofloxacin', 'Aldioxum', 'Amylocaine', 'Bambermycine', 'Bambermycinum', 'Bambermicina', 'Bambermycin', 'Barbexaclone', 'Boroglycerin', 'Buprenorfina', 'Buprenorphinum', 'Casanthranol', 'Cefepimum', 'Cefepima', 'Cetiedilum', 'Enclomiphene', 'Dantrolenum', 'Dantroleno', 'Dihexyverine', 'Elapegademase', 'Guaietolin', 'Halazepamum', 'Idrocodone', 'Levocabastinum', 'Levocabastina', 'Methorfinan', 'Levorfanolo', 'Mangafodipir', 'Bruinsteen', 'Braunstein', 'Mangandioxid', 'Mangaanbioxyde', 'Mangaandioxyde', 'Melarsomine', 'Metaraminolum', 'Acetilmetadol', 'Acetylmethadolum', 'Methyprylone', 'Methyprylonum', 'Metiprilona', 'Nifurzide', 'Nusinersen', 'Omoconazole', 'Orgotein', 'Orgotein', 'Orgotein', 'Orgotein', 'Orgotein', 'Orgotein', 'Methocidine', 'Methocidinum', 'Metocidina', 'Pegvaliase', 'Pergolidum', 'Pergolida', 'Profenamina', 'Ritodrina', 'Ritodrinium', 'Calcitonine', 'Calcitoninum', 'Secretine', 'Secretinum', 'Secretina', 'Sitosterols', 'Sucralfatum', 'Sucralfato', 'Tocophersola

In [346]:
df_traduccion = pd.DataFrame(lista_traduccion,columns=['Traduccion'])
df_traduccion = df_traduccion.rename_axis('Traduccion_Id').reset_index()
df_traduccion.duplicated()
df_traduccion


,Traduccion_Id,Traduccion
0,0,Alatrofloxacin
1,1,Aldioxum
2,2,Amylocaine
3,3,Bambermycine
4,4,Bambermycinum
...,...,...
63,63,Sitosterols
64,64,Sucralfatum
65,65,Sucralfato
66,66,Tocophersolan


In [349]:
df_sinonimo_ingrediente = pd.DataFrame(lista_ingrediente_sinonimo,columns=['Ingrediente_Sinonimo'])
df_sinonimo_ingrediente = df_sinonimo_ingrediente.rename_axis('Ingrediente_Sinonimo_Id').reset_index()
df_sinonimo_ingrediente.duplicated()
df_sinonimo_ingrediente

,Ingrediente_Sinonimo_Id,Ingrediente_Sinonimo
0,0,tert Butylurea
1,1,N tert Butylurea
2,2,Oleyl acetate
3,3,Osbond acid
4,4,Methanol dibutylamino
...,...,...
9449,9449,zomepirac
9450,9450,zonisamide
9451,9451,zopiclone
9452,9452,zotepine


In [350]:
df_ingredientes = pd.DataFrame(lista_ingredientes,columns=['Ingrediente'])
df_ingredientes = df_ingredientes.rename_axis('Ingrediente_Id').reset_index()
df_ingredientes.duplicated()
df_ingredientes

,Ingrediente_Id,Ingrediente
0,0,abacavir
1,1,abaloparatide
2,2,abarelix
3,3,abatacept
4,4,abciximab
...,...,...
8837,8837,zonisamide
8838,8838,zopiclone
8839,8839,zotepine
8840,8840,zucchini extract
